In [1]:
# Import Libraries
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [2]:
#Upload VGG16

vgg16 = VGG16(weights='imagenet')

In [3]:
vgg16.save('vgg16.h5')

In [4]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [5]:
CLASS_INDEX = None

In [6]:
CLASS_INDEX_PATH = 'C:\\Users\\avish\\Desktop\\Major2\\imagenet_class_index.json'
#CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

In [7]:
#Sorting the list in decreasing order of probability
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [8]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [9]:
#Getting car categories list to check validity of car
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('C:/Users/avish/Desktop/Major2/data1a/training/01-whole')
    for i, img_path in enumerate(img_list):
        img = prepare_image('C:/Users/avish/Desktop/Major2/data1a/training/01-whole/'+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [10]:
cat_counter=get_car_categories()

1/1 [==============================] - 2s 2s/step
0 / 920 complete
1/1 [==============================] - 0s 409ms/step
100 / 920 complete
1/1 [==============================] - 0s 397ms/step


1/1 [==============================] - 0s 395ms/step
200 / 920 complete
1/1 [==============================] - 0s 405ms/step
300 / 920 complete
1/1 [==============================] - 0s 461ms/step


1/1 [==============================] - 0s 383ms/step
400 / 920 complete
1/1 [==============================] - 0s 389ms/step


1/1 [==============================] - 0s 389ms/step
500 / 920 complete
1/1 [==============================] - 0s 400ms/step
600 / 920 complete
1/1 [==============================] - 0s 382ms/step


1/1 [==============================] - 0s 381ms/step
700 / 920 complete
1/1 [==============================] - 0s 374ms/step


1/1 [==============================] - 0s 389ms/step
800 / 920 complete
1/1 [==============================] - 0s 396ms/step
900 / 920 complete
1/1 [==============================] - 0s 385ms/step


In [11]:
with open('cat_counter.pk', 'wb') as f:
    pk.dump(cat_counter,f,-1)

In [12]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [13]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# First Check - car or not

In [14]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [15]:
car_categories_gate('C:/Users/avish/Desktop/Major2/download.jpg', cat_list)

1/1 [==============================] - 0s 392ms/step
Validating that this is a picture of your car...
('n03791053', 'motor_scooter')


'Validation complete - proceed to damage evaluation'

In [16]:
car_categories_gate('C:/Users/avish/Desktop/Major2/download2.jpg', cat_list)

1/1 [==============================] - 0s 407ms/step
Validating that this is a picture of your car...


'Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again.'